In [ ]:
import pandas as pd
df= pd.read_csv('../data/Clean_Dataset.csv', index_col=0)

In [ ]:
df.head()

![](../../imgs/featureEngin.png)

In [ ]:
df.airline.value_counts()

In [ ]:
df.source_city.value_counts()

In [ ]:
df.destination_city.value_counts()

In [ ]:
df.departure_time.value_counts()

In [ ]:
df.arrival_time.value_counts()

In [ ]:
df.stops.value_counts()

In [ ]:
df["class"].value_counts()

In [ ]:
# describe the  of  duration of the flights

df.duration.describe().T[["min", "max", "mean"]]

In [ ]:
# prepross dataframe 
df.drop(["flight"], axis=1, inplace=True)

In [ ]:
# binary encoding 
df["class"]= df["class"].apply(lambda x: 1 if x=="Economy" else 0)

In [ ]:
# encode "stops" features 
df["stops"].unique()
# df["stops"]= df["stops"].apply(lambda x: 0 if x=="non-stop" else 1
pd.factorize(df["stops"])

In [ ]:
df.stops = pd.factorize(df["stops"])[0]

In [ ]:
df.sample(5, random_state=46)

In [ ]:
# onrhot encpding  of airine  feature of  our datsset 
pd.get_dummies(df, columns=["airline"], drop_first=True, prefix="AirL", dtype=int).head()

In [ ]:
cat_features = df.select_dtypes(include="object").columns
cat_features

In [ ]:
# onehot encode all object type features
# diaplay all features columns  ( max columns)
pd.set_option('display.max_columns', None)

pd.get_dummies(df, columns=cat_features, drop_first=True, prefix=cat_features, dtype=int).head()

In [ ]:
# we can proceed more  by onehot encoding all categorical features of our dataset

df= pd.get_dummies(df, drop_first=True ,  dtype=int)
df.head()

![](../../imgs/model.png)

## LinearRegression

![](../../imgs/dataSplit.jpg)

In [ ]:
X= df.drop("price", axis=1)
y= df.price

In [ ]:
# import a randomforest regressor from sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=46)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
reg= RandomForestRegressor(n_estimators=50, random_state=46, n_jobs=-1)
reg.fit(X_train, y_train)


![](../../imgs/metrics.png)

In [ ]:
# print main  metrics  score of the model
reg.score(X_test, y_test)

In [ ]:

# import R2 , mean_squared_error  and mean absolute error from sklearn.metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
y_pred=reg.predict(X_test)
print(f"    R2   {r2_score(y_test, y_pred):.3f}")
print(f"    Mean Squared Error MSE :  {mean_squared_error(y_test, y_pred):.3f}")
print(f"    MAE  :  {mean_absolute_error(y_test, y_pred):.3f}")
print(f"    RMSE  :  {math.sqrt(mean_squared_error(y_test, y_pred)):.3f}")


In [ ]:
# plot  y_test vs y_pred
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, c="lightcoral")
plt.xlabel("y_test")
plt.ylabel("y_pred")
plt.title("y_test vs y_pred")
plt.show()

In [ ]:
df.price.describe().T# describe the price of the flights

In [ ]:
# display most important 5 features of the model

import numpy as np
importances = reg.feature_importances_
indices = np.argsort(importances)[::-1]
names = [X.columns[i] for i in indices]
plt.figure(figsize=(10, 5))
plt.title("Top 5 Feature Importance")
plt.bar(range(5), importances[indices[:5]])
plt.xticks(range(5), names[:5], rotation=45, ha="right") # Rotate labels for better readability
plt.tight_layout()
plt.show()


# GridSearch 

In [ ]:
# proceed through gridseachcv
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150, 200], # The number of trees in the forest
    'max_features': ['auto', 'sqrt'], # The number of features to consider when looking for the best split
    'max_depth': [None, 10, 20, 30,], # The maximum depth of the tree
    'min_samples_split': [2, 5, 10], # The minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4] # The minimum number of samples required to be at a leaf node
}

grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


In [ ]:
# best param
grid_search.best_params_

## RandomizeGridSearch 

In [ ]:
# import randomizeserachcv 
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'n_estimators': [50, 100, 150, 200], # The number of trees in the forest
    'max_features': ['auto', 'sqrt'], # The number of features to consider when looking for the best split
    'max_depth': [None, 10, 20, 30,], # The maximum depth of the tree
    'min_samples_split': [2, 5, 10], # The minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4] # The minimum number of samples required to be at a leaf node
}

random_search = RandomizedSearchCV(estimator=reg, param_distributions=param_grid, n_iter=100, cv=5, n_jobs=-1, verbose=2)
random_search.fit(X_train, y_train)
